In [3]:
import os
import base64
import json
from pathlib import Path
import requests
from PIL import Image
import time

In [11]:
class HandwritingOCR:
    def __init__(self, api_key, prompt):
        self.api_key = api_key
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        self.api_url = 'https://chat.int.bayer.com/api/v2/chat/completions'
        self.prompt = prompt
    
    def encode_image_to_base64(self, image_path):
        """画像をbase64エンコード"""
        try:
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
        except Exception as e:
            print(f"画像エンコードエラー: {image_path} - {e}")
            return None
    
    def resize_image_if_needed(self, image_path, max_size=(1024, 1024)):
        """画像サイズが大きい場合リサイズ"""
        try:
            with Image.open(image_path) as img:
                if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
                    img.thumbnail(max_size, Image.Resampling.LANCZOS)
                    resized_path = f"resized_{Path(image_path).name}"
                    img.save(resized_path, quality=85)
                    return resized_path
                return image_path
        except Exception as e:
            print(f"画像リサイズエラー: {image_path} - {e}")
            return image_path
    
    def extract_text_from_image(self, image_path):
        """単一画像からテキスト抽出"""
        # 必要に応じて画像をリサイズ
        processed_image_path = self.resize_image_if_needed(image_path)
        
        # base64エンコード
        base64_image = self.encode_image_to_base64(processed_image_path)
        if not base64_image:
            return None
        
        # APIリクエストのペイロード
        payload = {
            "model": "gemini-2.5-pro-preview-05-06", 
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": self.prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"  # 高解像度で処理
                            }
                        }
                    ]
                }
            ],
            #"max_tokens": 1000,
            "temperature": 0  # 一貫性のため低く設定
        }
        
        try:
            response = requests.post(self.api_url, headers=self.headers, json=payload)
            response.raise_for_status()
            
            result = response.json()
            extracted_text = result['choices'][0]['message']['content']
            
            # リサイズした一時ファイルを削除
            if processed_image_path != image_path and os.path.exists(processed_image_path):
                os.remove(processed_image_path)
            
            return extracted_text
            
        except requests.exceptions.RequestException as e:
            print(f"API呼び出しエラー: {e}")
            return None
        except Exception as e:
            print(f"予期しないエラー: {e}")
            return None
    
    def process_multiple_images(self, image_directory, output_file="extracted_text.json"):
        """複数画像を処理"""
        # サポートする画像形式
        supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
        
        # 画像ファイルを取得
        image_files = set()
        for ext in supported_formats:
            image_files.update(Path(image_directory).glob(f"*{ext}"))
            image_files.update(Path(image_directory).glob(f"*{ext.upper()}"))
        
        image_files = sorted(image_files)
        print(f"処理対象: {len(image_files)}枚の画像")
        
        results = {}
        
        for i, image_path in enumerate(image_files, 1):
            print(f"処理中 ({i}/{len(image_files)}): {image_path.name}")
            
            extracted_text = self.extract_text_from_image(str(image_path))
            
            if extracted_text:
                results[image_path.name] = {
                    "file_path": str(image_path),
                    "extracted_text": extracted_text,
                    "status": "success"
                }
                print(f"✓ 完了: {image_path.name}")

                print("次の処理を待っています。")
                time.sleep(20)
            else:
                results[image_path.name] = {
                    "file_path": str(image_path),
                    "extracted_text": "",
                    "status": "failed"
                }
                print(f"✗ 失敗: {image_path.name}")

                print("次の処理を待っています。")
                time.sleep(20)
        
        # 結果をJSONファイルに保存
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        print(f"\n結果を{output_file}に保存しました")
        
        return results

In [ ]:
API_KEY = "Your API"

PROMPT = """
# 手書き文字抽出・CSV化プロンプト（改良版）

## 背景・目的
Workshopで「もっと職場をよくするには？」というテーマで参加者が書いた手書きの意見を、漏れなく正確に抽出してCSV形式で整理する。

## 実行手順

### 1. 画像解析の準備
- 画像を明瞭に確認し、手書き文字の判読可能性を評価
- 不鮮明な部分は「判読困難」として明記
- 複数の意見が1枚に書かれている場合はすべて抽出

### 2. 文字読み取りの実行
以下の順序で実行：
1. **全体把握**：画像全体をスキャンし、書かれている項目数を把握
2. **詳細読み取り**：各項目を個別に丁寧に読み取り
3. **確認作業**：読み取り結果を再度画像と照合

### 3. カテゴリー分類基準
以下の基準でカテゴリーを判定：
- **コミュニケーション改善**：会話、情報共有、チームワーク関連
- **業務効率化**：プロセス改善、ツール導入、時短関連
- **職場環境**：物理環境、設備、レイアウト関連
- **人事・制度**：評価制度、研修、福利厚生関連
- **マネジメント**：リーダーシップ、意思決定、組織運営関連
- **その他**：上記に該当しない項目

### 4. 出力要件

#### CSV形式：
```
ファイル名,カテゴリー,項目,補足情報
```

#### 出力ルール：
- **ファイル名**：画像ファイル名をそのまま記載
- **カテゴリー**：上記6分類のいずれかを必ず選択
- **項目**：読み取った手書き文字をそのまま記載（推測による補完は避ける）
- **補足情報**：判読困難な文字、推測部分、特記事項を記載

#### 品質確保のポイント：
1. **完全性**：画像内のすべての文字を抽出（見落とし厳禁）
2. **正確性**：推測ではなく読み取れる範囲で記載
3. **一貫性**：カテゴリー分類基準を全画像で統一適用
4. **透明性**：不明瞭な部分は「？」や「[判読困難]」で明記

## 処理完了後の確認事項
- 抽出項目数が画像内の実際の項目数と一致しているか
- カテゴリー分類に偏りや分類ミスがないか
- 判読困難部分が適切にマークされているか

## エラー処理
- 完全に判読不可能な画像：「画像不鮮明により抽出不可」と記載
- 部分的に判読困難：読み取れる部分のみ抽出し、困難部分を明記
"""


ocr = HandwritingOCR(API_KEY, prompt=PROMPT)

# 画像ディレクトリのパスを指定
image_directory = "Directory where photos are saved."  # 画像が保存されているディレクトリ

# 複数画像を処理
results = ocr.process_multiple_images(image_directory, "handwriting_results.json")

# 結果の概要表示
successful = sum(1 for r in results.values() if r['status'] == 'success')
failed = len(results) - successful

print(f"\n=== 処理結果 ===")
print(f"成功: {successful}枚")
print(f"失敗: {failed}枚")

# 成功した結果のテキストを統合表示
print(f"\n=== 抽出されたテキスト ===")
for filename, result in results.items():
    if result['status'] == 'success':
        print(f"\n[{filename}]")
        print(result['extracted_text'])
        print("-" * 50)

処理対象: 55枚の画像
処理中 (1/55): IMG_0820.jpg
✓ 完了: IMG_0820.jpg
次の処理を待っています。
処理中 (2/55): IMG_0821.jpg
✓ 完了: IMG_0821.jpg
次の処理を待っています。
処理中 (3/55): IMG_0822.jpg
✓ 完了: IMG_0822.jpg
次の処理を待っています。
処理中 (4/55): IMG_0823.jpg
✓ 完了: IMG_0823.jpg
次の処理を待っています。
処理中 (5/55): IMG_0824.jpg
✓ 完了: IMG_0824.jpg
次の処理を待っています。
処理中 (6/55): IMG_0825.jpg
✓ 完了: IMG_0825.jpg
次の処理を待っています。
処理中 (7/55): IMG_0826.jpg
✓ 完了: IMG_0826.jpg
次の処理を待っています。
処理中 (8/55): IMG_0827.jpg
✓ 完了: IMG_0827.jpg
次の処理を待っています。
処理中 (9/55): IMG_0828.jpg
✓ 完了: IMG_0828.jpg
次の処理を待っています。
処理中 (10/55): IMG_0829.jpg
✓ 完了: IMG_0829.jpg
次の処理を待っています。
処理中 (11/55): IMG_0830.jpg
✓ 完了: IMG_0830.jpg
次の処理を待っています。
処理中 (12/55): IMG_0831.jpg
✓ 完了: IMG_0831.jpg
次の処理を待っています。
処理中 (13/55): IMG_0832.jpg
✓ 完了: IMG_0832.jpg
次の処理を待っています。
処理中 (14/55): IMG_0833.jpg
✓ 完了: IMG_0833.jpg
次の処理を待っています。
処理中 (15/55): IMG_0834.jpg
✓ 完了: IMG_0834.jpg
次の処理を待っています。
処理中 (16/55): IMG_0835.jpg
✓ 完了: IMG_0835.jpg
次の処理を待っています。
処理中 (17/55): IMG_0836.jpg
✓ 完了: IMG_0836.jpg
次の処理を待っています。
処理中 (18/55

In [39]:
import pandas as pd

def save_to_excel_pandas(data, output_path='extracted_texts.xlsx'):
    """
    data: 辞書型。キーが画像ファイル名、値は {'extracted_text': ...} の形式を想定。
    output_path: 保存先のパス（.xlsx で終わるようにする）。
    """
    # ① DataFrame の中身を作成
    df_data = []
    for file_key, item in data.items():
        df_data.append({
            'ファイル名': file_key,
            'extracted_text': item.get('extracted_text', '')
        })

    df = pd.DataFrame(df_data)

    # ② Excel 形式で保存（インデックス不要）
    df.to_excel(output_path, index=False)
    print(f"Excelファイル `{output_path}` に保存しました。")
    print(f"保存された行数: {len(df)}")


In [ ]:
output = "Output file path"

save_to_excel_pandas(results, output_path=output)

Excelファイル `C:\Users\GPGTT\OneDrive - Bayer\Desktop\extracted_texts.xlsx` に保存しました。
保存された行数: 55
